In [9]:
# Fundamental classes
import numpy as np 
import pandas as pd 
import tensorflow as tf
import os
from tqdm import tqdm

# Image related
import cv2
from PIL import Image

#Performance Plot
from sklearn import metrics 
from sklearn.metrics import f1_score, accuracy_score,confusion_matrix,classification_report
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# For the model and it's training
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout

# Time
import time
import datetime

In [2]:
# date_time function

def date_time(x):
    if x==1:
        return 'Timestamp: {:%Y-%m-%d %H:%M:%S}'.format(datetime.datetime.now())
    if x==2:    
        return 'Timestamp: {:%Y-%b-%d %H:%M:%S}'.format(datetime.datetime.now())
    if x==3:  
        return 'Date now: %s' % datetime.datetime.now()
    if x==4:  
        return 'Date today: %s' % datetime.date.today()

In [3]:
def plot_performance(history=None, figure_directory=None, ylim_pad=[0, 0]):
    xlabel = 'Epoch'
    legends = ['Training', 'Validation']

    plt.figure(figsize=(20, 5))

    y1 = history.history['accuracy']
    y2 = history.history['val_accuracy']

    min_y = min(min(y1), min(y2))-ylim_pad[0]
    max_y = max(max(y1), max(y2))+ylim_pad[0]


    plt.subplot(121)

    plt.plot(y1)
    plt.plot(y2)

    plt.title('Model Accuracy\n'+date_time(1), fontsize=17)
    plt.xlabel(xlabel, fontsize=15)
    plt.ylabel('Accuracy', fontsize=15)
    plt.ylim(min_y, max_y)
    plt.legend(legends, loc='upper left')
    plt.grid()

    y1 = history.history['loss']
    y2 = history.history['val_loss']

    min_y = min(min(y1), min(y2))-ylim_pad[1]
    max_y = max(max(y1), max(y2))+ylim_pad[1]


    plt.subplot(122)

    plt.plot(y1)
    plt.plot(y2)

    plt.title('Model Loss\n'+date_time(1), fontsize=17)
    plt.xlabel(xlabel, fontsize=15)
    plt.ylabel('Loss', fontsize=15)
    plt.ylim(min_y, max_y)
    plt.legend(legends, loc='upper left')
    plt.grid()
    if figure_directory:
        plt.savefig(figure_directory+"/history")

    plt.show()

In [37]:
def load_images_from_dir(folder, label_file):
    pathtodir = os.getcwd()
    pathtodir = os.path.join(pathtodir, folder)
    print(pathtodir)

    aux_imagini = []
    intervale = np.linspace(start=0, stop=255, num=15)
    cnt = 0
    for nume_imagine in tqdm(label_file.loc[:, "image_id"]):
        image_name = nume_imagine + ".png"
        pathToImage = os.path.join(pathtodir, image_name)
        imagine = np.array(Image.open(pathToImage))
        imagine = np.ravel(imagine.copy())
        if(imagine.shape == (6400, )):
            # if folder == "train":#in viitor va trb sa sterg imaginile alb/negru
            #     continue
            imagine = np.concatenate((imagine, imagine, imagine))
        imagine = np.digitize(imagine, intervale)
        img = imagine.copy()
        imagine = np.reshape(img, (80,80,3))
        aux_imagini.append(imagine)
        cnt += 1
    imagini = np.array(aux_imagini)
    return imagini
    

In [38]:
train_labels = pd.read_csv("train.csv")
test_labels = pd.read_csv("test.csv")
validation_labels = pd.read_csv("validation.csv")

train_images = load_images_from_dir("train", train_labels)
test_images = load_images_from_dir("test", test_labels)
validation_images = load_images_from_dir("validation", validation_labels)

C:\Users\Fabi\Desktop\ml\kaggle\Realistic Image Classification @ UnivBuc\train


100%|██████████████████████████████████████████████████████████████████████████| 10500/10500 [00:09<00:00, 1066.55it/s]


C:\Users\Fabi\Desktop\ml\kaggle\Realistic Image Classification @ UnivBuc\test


100%|████████████████████████████████████████████████████████████████████████████| 4500/4500 [00:03<00:00, 1167.88it/s]


C:\Users\Fabi\Desktop\ml\kaggle\Realistic Image Classification @ UnivBuc\validation


100%|████████████████████████████████████████████████████████████████████████████| 3000/3000 [00:02<00:00, 1178.98it/s]


In [39]:
# Building the model
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu', input_shape=train_images.shape[1:]))
model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(43, activation='softmax'))

# Compilation of the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

#M odel display
model.summary()

C:\Users\Fabi\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_8 (Conv2D)                    │ (None, 76, 76, 32)          │           2,432 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_9 (Conv2D)                    │ (None, 72, 72, 32)          │          25,632 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_4 (MaxPooling2D)       │ (None, 36, 36, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_6 (Dropout)                  │ (None, 36, 36, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_10 (Conv2D)                   │ (None, 34, 34, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_11 (Conv2D)                   │ (None, 32, 32, 64)          │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_5 (MaxPooling2D)       │ (None, 16, 16, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_7 (Dropout)                  │ (None, 16, 16, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_2 (Flatten)                  │ (None, 16384)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 256)                 │       4,194,560 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_8 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 43)                  │          11,051 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,289,099 (16.36 MB)

 Trainable params: 4,289,099 (16.36 MB)

 Non-trainable params: 0 (0.00 B)

In [41]:

train_images = to_categorical(train_images, 16)
validation_images = to_categorical(validation_images, 16)

with tf.device('/GPU:0'):
    epochs = 25
    history1 = model.fit(train_images, train_labels, batch_size=32, epochs=epochs, validation_data=(validation_images, validation_labels))

MemoryError: Unable to allocate 39.1 GiB for an array with shape (10500, 80, 80, 3, 26) and data type int64